# Mood Trend Prediction

A notebook that will take your mood data from Daylio app and make predictions based on it using a machine learning model.

## Problem Statement

**Objective:** To predict what mood would the user be for the next days. <br> A dashboard summarizing this can also help which is focused on people with Bipolar Disorder.

**Performance Measure:** RMSE and MAE

**Assumption:** This is a regression problem because my data is daily. I need to get the scale of my mood each day and predict patterns to help me with my bipolar disorder.

## Data Preparation

In [ ]:
# get the data from daylio app
import pandas as pd
import numpy as np



In [ ]:
# look at the data structure
# head
# info
# value_counts
# describe
# plot a histogram for each numerical attribute

In [ ]:
# create a test set

## Exploratory Data Analysis

In [ ]:
# make a copy of the original

In [ ]:
# visualize data

In [ ]:
# look for correlatiosn

In [ ]:
# experiment with attribute combinations

## Data Preprocessing

1. Get rid of the corresponding districts. <br>
2. Get rid of the whole attribute.<br>
3. Set the missing values to some value (zero, the mean, the median, etc.).<br>
This is called imputation.

In [ ]:
# clean the data


In [ ]:
# handle text and categorical attributes

In [ ]:
# feature scaling and transformation

In [ ]:
# transformation pipelines

## Select and Train a Model

In [ ]:
# use pycaret for model selection ha

In [ ]:
# train and evaluate on the training set

In [ ]:
# better evaluation using cross-validation

## Fine Tuning

In [ ]:
# grid search

In [ ]:
# analyze best models and their errors

In [ ]:
# evaluate your model on the test set